# Seyed Mohammad Amin Atyabi - 830402014 - HW 3

In [1]:
def evaluate_expression(formula, value):
    expression = formula
    for v in value.axes[-1].values:
        expression = expression.replace(f'_{str(v)}_', str(value[v]))
    return eval(expression)


def train_model(data):
    P = data.loc[data['y'] == True]
    h = 'False'
    while len(P) != 0:
        r = 'True'
        r_features = set()
        N = data.loc[data['y'] == False]
        while len(N) != 0:
            if set(data.axes[-1].values).issubset(r_features):
                break
            vj = {}
            for f in data.axes[-1].values:
                if f == 'y':
                    continue
                if f not in r_features:
                    n_plus = 0
                    for i, p in P.iterrows():
                        if p['y'] == evaluate_expression(r + f' and _{f}_', p):
                            n_plus += 1
                    n_minus = 0
                    for i, n in N.iterrows():
                        if n['y'] == evaluate_expression(r + f' and _{f}_', n):
                            n_minus += 1
                    heuristic = n_plus / max(float(n_minus), 0.0001)
                    vj[f] = heuristic
            fj = sorted(vj.items(), key=lambda item: item[1])[-1][0]
            r_features.add(fj)
            n_drop = []
            for i, n in N.iterrows():
                if not n[fj]:
                    n_drop.append(i)
            if len(n_drop) > 0:
                r = r + f' and _{fj}_'
                N = N.drop(index=n_drop, inplace=False)
        h = h + f' or ({r})'
        covered = []
        for i, p in P.iterrows():
            if p['y'] == evaluate_expression(r, p):
                covered.append(i)
        if len(covered) == 0:
            break
        P = P.drop(index=covered, inplace=False)
    return h


def evaluate_model(data, model):
    count = 0
    for _, d in data.iterrows():
        res = (evaluate_expression(model, d) == d['y'])
        if not res:
            count += 1
    return count


def latex_expression(model):
    return model.replace('False or ', '').replace('True and ', '') \
        .replace('_f', 'f_{').replace('_ ', '} ').replace('_)', '})') \
        .replace('and', r'\wedge').replace('or', r'\vee')

In [2]:
import pandas
from scipy import io as sio
from sklearn.model_selection import train_test_split

data = pandas.DataFrame(sio.loadmat('Data/DNFData.mat').get('Data'))

columns = []
for i in range(len(data.iloc[-1]) - 1):
    columns.append(f'f{i + 1}')
columns.append('y')

data = data.astype(bool)
data = pandas.DataFrame(data.values, columns=columns)

train, test = train_test_split(data, test_size=0.5)

In [ ]:
from IPython.display import display, Markdown

samples = list(range(20, 1001, 20))
errors = []

for s in samples:
    model = train_model(train.sample(s))
    error = evaluate_model(train, model)
    errors.append(error)
    display(Markdown(f"""
    Number of Samples = {s}
    Error = {error}
    $$ {latex_expression(model)} $$
    """))
    if error == 0:
        display(Markdown(f"""
        *Best model*
        
        {model}
        """))


    Number of Samples = 20
    Error = 46
    $$ (f_{58} \wedge f_{30} \wedge f_{29} \wedge f_{19} \wedge f_{21} \wedge f_{42} \wedge f_{6} \wedge f_{92} \wedge f_{94} \wedge f_{99} \wedge f_{100}) $$
    


    Number of Samples = 40
    Error = 46
    $$ (f_{94} \wedge f_{84} \wedge f_{59} \wedge f_{99} \wedge f_{71} \wedge f_{74} \wedge f_{96} \wedge f_{97} \wedge f_{87} \wedge f_{95} \wedge f_{100}) $$
    


    Number of Samples = 60
    Error = 44
    $$ (f_{71} \wedge f_{4} \wedge f_{59} \wedge f_{66} \wedge f_{94} \wedge f_{51} \wedge f_{96} \wedge f_{97} \wedge f_{90} \wedge f_{84} \wedge f_{98}) \vee (f_{76} \wedge f_{71} \wedge f_{70} \wedge f_{63} \wedge f_{97} \wedge f_{66} \wedge f_{53} \wedge f_{75} \wedge f_{95} \wedge f_{100} \wedge f_{91} \wedge f_{99}) \vee (f_{76} \wedge f_{71} \wedge f_{70} \wedge f_{63} \wedge f_{23} \wedge f_{97} \wedge f_{66} \wedge f_{96} \wedge f_{89} \wedge f_{88} \wedge f_{99} \wedge f_{98}) $$
    


    Number of Samples = 80
    Error = 42
    $$ (f_{20} \wedge f_{16} \wedge f_{39} \wedge f_{62} \wedge f_{96} \wedge f_{95} \wedge f_{83} \wedge f_{70} \wedge f_{67} \wedge f_{69} \wedge f_{93} \wedge f_{91} \wedge f_{90}) \vee (f_{20} \wedge f_{16} \wedge f_{39} \wedge f_{11} \wedge f_{62} \wedge f_{100} \wedge f_{96} \wedge f_{71} \wedge f_{28} \wedge f_{95} \wedge f_{98} \wedge f_{90} \wedge f_{99}) \vee (f_{20} \wedge f_{16} \wedge f_{41} \wedge f_{37} \wedge f_{77} \wedge f_{83} \wedge f_{79} \wedge f_{92} \wedge f_{99} \wedge f_{86} \wedge f_{97} \wedge f_{94}) \vee (f_{30} \wedge f_{63} \wedge f_{14} \wedge f_{19} \wedge f_{64} \wedge f_{51} \wedge f_{47} \wedge f_{6} \wedge f_{91} \wedge f_{92} \wedge f_{98} \wedge f_{93} \wedge f_{82}) $$
    


    Number of Samples = 100
    Error = 43
    $$ (f_{77} \wedge f_{1} \wedge f_{68} \wedge f_{93} \wedge f_{76} \wedge f_{87} \wedge f_{16} \wedge f_{89} \wedge f_{55} \wedge f_{94} \wedge f_{40} \wedge f_{70} \wedge f_{99} \wedge f_{85}) \vee (f_{20} \wedge f_{14} \wedge f_{39} \wedge f_{64} \wedge f_{65} \wedge f_{68} \wedge f_{43} \wedge f_{91} \wedge f_{94} \wedge f_{98} \wedge f_{97} \wedge f_{100}) \vee (f_{87} \wedge f_{76} \wedge f_{53} \wedge f_{63} \wedge f_{93} \wedge f_{77} \wedge f_{71} \wedge f_{1} \wedge f_{31} \wedge f_{27} \wedge f_{96} \wedge f_{74} \wedge f_{86} \wedge f_{80}) $$
    

...


    Number of Samples = 240
    Error = 34
    $$ (f_{4} \wedge f_{5} \wedge f_{2} \wedge f_{48} \wedge f_{52} \wedge f_{85} \wedge f_{61} \wedge f_{54} \wedge f_{86} \wedge f_{93} \wedge f_{63} \wedge f_{43} \wedge f_{84} \wedge f_{30}) \vee (f_{4} \wedge f_{97} \wedge f_{39} \wedge f_{16} \wedge f_{5} \wedge f_{80} \wedge f_{26} \wedge f_{78} \wedge f_{14} \wedge f_{65} \wedge f_{91} \wedge f_{50} \wedge f_{88} \wedge f_{100} \wedge f_{99} \wedge f_{94} \wedge f_{93}) \vee (f_{4} \wedge f_{97} \wedge f_{46} \wedge f_{56} \wedge f_{91} \wedge f_{75} \wedge f_{33} \wedge f_{86} \wedge f_{65} \wedge f_{64} \wedge f_{99} \wedge f_{90} \wedge f_{74} \wedge f_{98} \wedge f_{100} \wedge f_{96}) \vee (f_{4} \wedge f_{97} \wedge f_{46} \wedge f_{55} \wedge f_{64} \wedge f_{27} \wedge f_{91} \wedge f_{28} \wedge f_{95} \wedge f_{22} \wedge f_{85} \wedge f_{89} \wedge f_{98}) \vee (f_{4} \wedge f_{5} \wedge f_{2} \wedge f_{86} \wedge f_{65} \wedge f_{43} \wedge f_{68} \wedge f_{52} \wedge f_{87} \wedge f_{74} \wedge f_{39} \wedge f_{63} \wedge f_{93} \wedge f_{79} \wedge f_{96}) \vee (f_{4} \wedge f_{97} \wedge f_{16} \wedge f_{39} \wedge f_{5} \wedge f_{21} \wedge f_{87} \wedge f_{65} \wedge f_{6} \wedge f_{94} \wedge f_{76} \wedge f_{86} \wedge f_{99} \wedge f_{85} \wedge f_{93}) \vee (f_{4} \wedge f_{90} \wedge f_{5} \wedge f_{13} \wedge f_{48} \wedge f_{41} \wedge f_{53} \wedge f_{91} \wedge f_{71} \wedge f_{73} \wedge f_{45} \wedge f_{95} \wedge f_{85} \wedge f_{92}) \vee (f_{97} \wedge f_{96} \wedge f_{94} \wedge f_{4} \wedge f_{90} \wedge f_{16} \wedge f_{80} \wedge f_{86} \wedge f_{84} \wedge f_{98} \wedge f_{74} \wedge f_{77} \wedge f_{83} \wedge f_{93}) \vee (f_{68} \wedge f_{36} \wedge f_{16} \wedge f_{78} \wedge f_{96} \wedge f_{22} \wedge f_{54} \wedge f_{32} \wedge f_{25} \wedge f_{85} \wedge f_{79} \wedge f_{100} \wedge f_{99} \wedge f_{98}) \vee (f_{86} \wedge f_{68} \wedge f_{11} \wedge f_{90} \wedge f_{69} \wedge f_{35} \wedge f_{4} \wedge f_{93} \wedge f_{33} \wedge f_{97} \wedge f_{78} \wedge f_{61} \wedge f_{99} \wedge f_{96}) $$
    


    Number of Samples = 260
    Error = 31
    $$ (f_{16} \wedge f_{95} \wedge f_{97} \wedge f_{98} \wedge f_{6} \wedge f_{8} \wedge f_{68} \wedge f_{78} \wedge f_{52} \wedge f_{81} \wedge f_{67} \wedge f_{48} \wedge f_{87} \wedge f_{100} \wedge f_{96}) \vee (f_{5} \wedge f_{4} \wedge f_{84} \wedge f_{35} \wedge f_{90} \wedge f_{33} \wedge f_{59} \wedge f_{22} \wedge f_{14} \wedge f_{67} \wedge f_{95} \wedge f_{48} \wedge f_{68} \wedge f_{86}) \vee (f_{16} \wedge f_{95} \wedge f_{34} \wedge f_{90} \wedge f_{97} \wedge f_{43} \wedge f_{48} \wedge f_{45} \wedge f_{99} \wedge f_{96} \wedge f_{55} \wedge f_{40} \wedge f_{98} \wedge f_{84} \wedge f_{100}) \vee (f_{16} \wedge f_{93} \wedge f_{5} \wedge f_{77} \wedge f_{69} \wedge f_{15} \wedge f_{78} \wedge f_{20} \wedge f_{88} \wedge f_{87} \wedge f_{91} \wedge f_{60} \wedge f_{95} \wedge f_{89}) \vee (f_{16} \wedge f_{93} \wedge f_{100} \wedge f_{73} \wedge f_{99} \wedge f_{88} \wedge f_{96} \wedge f_{79} \wedge f_{97} \wedge f_{66} \wedge f_{46} \wedge f_{50} \wedge f_{72} \wedge f_{98}) \vee (f_{65} \wedge f_{34} \wedge f_{99} \wedge f_{100} \wedge f_{29} \wedge f_{57} \wedge f_{98} \wedge f_{82} \wedge f_{4} \wedge f_{13} \wedge f_{33} \wedge f_{58} \wedge f_{94} \wedge f_{93}) \vee (f_{27} \wedge f_{43} \wedge f_{22} \wedge f_{56} \wedge f_{51} \wedge f_{16} \wedge f_{61} \wedge f_{2} \wedge f_{63} \wedge f_{54} \wedge f_{99} \wedge f_{97} \wedge f_{95} \wedge f_{80} \wedge f_{100}) \vee (f_{93} \wedge f_{20} \wedge f_{39} \wedge f_{4} \wedge f_{14} \wedge f_{49} \wedge f_{62} \wedge f_{80} \wedge f_{5} \wedge f_{68} \wedge f_{77} \wedge f_{86}) $$
    


    Number of Samples = 280
    Error = 33
    $$ (f_{4} \wedge f_{41} \wedge f_{94} \wedge f_{13} \wedge f_{12} \wedge f_{71} \wedge f_{63} \wedge f_{69} \wedge f_{90} \wedge f_{92} \wedge f_{98} \wedge f_{97} \wedge f_{100} \wedge f_{99} \wedge f_{96}) \vee (f_{4} \wedge f_{41} \wedge f_{94} \wedge f_{13} \wedge f_{84} \wedge f_{71} \wedge f_{80} \wedge f_{10} \wedge f_{93} \wedge f_{34} \wedge f_{65} \wedge f_{98} \wedge f_{73}) \vee (f_{97} \wedge f_{100} \wedge f_{99} \wedge f_{52} \wedge f_{29} \wedge f_{17} \wedge f_{2} \wedge f_{24} \wedge f_{90} \wedge f_{86} \wedge f_{31} \wedge f_{76} \wedge f_{98} \wedge f_{96} \wedge f_{95}) \vee (f_{4} \wedge f_{97} \wedge f_{100} \wedge f_{25} \wedge f_{64} \wedge f_{98} \wedge f_{49} \wedge f_{71} \wedge f_{22} \wedge f_{94} \wedge f_{65} \wedge f_{91} \wedge f_{81} \wedge f_{86}) \vee (f_{54} \wedge f_{4} \wedge f_{99} \wedge f_{46} \wedge f_{85} \wedge f_{97} \wedge f_{92} \wedge f_{93} \wedge f_{68} \wedge f_{57} \wedge f_{87} \wedge f_{86} \wedge f_{94} \wedge f_{89}) \vee (f_{12} \wedge f_{96} \wedge f_{30} \wedge f_{45} \wedge f_{97} \wedge f_{100} \wedge f_{99} \wedge f_{4} \wedge f_{95} \wedge f_{89} \wedge f_{85} \wedge f_{98}) \vee (f_{2} \wedge f_{96} \wedge f_{12} \wedge f_{52} \wedge f_{41} \wedge f_{97} \wedge f_{34} \wedge f_{62} \wedge f_{90} \wedge f_{38} \wedge f_{48} \wedge f_{64} \wedge f_{91} \wedge f_{98} \wedge f_{99} \wedge f_{100}) \vee (f_{2} \wedge f_{13} \wedge f_{57} \wedge f_{54} \wedge f_{12} \wedge f_{49} \wedge f_{93} \wedge f_{47} \wedge f_{55} \wedge f_{90} \wedge f_{87} \wedge f_{92} \wedge f_{84} \wedge f_{85} \wedge f_{96}) \vee (f_{34} \wedge f_{94} \wedge f_{32} \wedge f_{99} \wedge f_{35} \wedge f_{97} \wedge f_{58} \wedge f_{11} \wedge f_{96} \wedge f_{84} \wedge f_{80} \wedge f_{59} \wedge f_{100} \wedge f_{98}) $$
    


    Number of Samples = 300
    Error = 36
    $$ (f_{28} \wedge f_{4} \wedge f_{52} \wedge f_{91} \wedge f_{64} \wedge f_{25} \wedge f_{39} \wedge f_{96} \wedge f_{2} \wedge f_{72} \wedge f_{10} \wedge f_{97} \wedge f_{65} \wedge f_{81} \wedge f_{90} \wedge f_{89}) \vee (f_{48} \wedge f_{5} \wedge f_{52} \wedge f_{4} \wedge f_{2} \wedge f_{94} \wedge f_{86} \wedge f_{85} \wedge f_{44} \wedge f_{57} \wedge f_{72} \wedge f_{93} \wedge f_{66} \wedge f_{98}) \vee (f_{95} \wedge f_{73} \wedge f_{84} \wedge f_{16} \wedge f_{98} \wedge f_{88} \wedge f_{100} \wedge f_{15} \wedge f_{72} \wedge f_{45} \wedge f_{93} \wedge f_{55} \wedge f_{99} \wedge f_{97} \wedge f_{96}) \vee (f_{95} \wedge f_{68} \wedge f_{17} \wedge f_{11} \wedge f_{2} \wedge f_{21} \wedge f_{91} \wedge f_{84} \wedge f_{70} \wedge f_{82} \wedge f_{100} \wedge f_{78} \wedge f_{48}) \vee (f_{95} \wedge f_{98} \wedge f_{1} \wedge f_{8} \wedge f_{17} \wedge f_{68} \wedge f_{84} \wedge f_{20} \wedge f_{100} \wedge f_{77} \wedge f_{74} \wedge f_{48} \wedge f_{99} \wedge f_{97} \wedge f_{96}) \vee (f_{27} \wedge f_{56} \wedge f_{91} \wedge f_{7} \wedge f_{95} \wedge f_{67} \wedge f_{73} \wedge f_{78} \wedge f_{100} \wedge f_{57} \wedge f_{75} \wedge f_{98} \wedge f_{89} \wedge f_{99} \wedge f_{96}) \vee (f_{27} \wedge f_{63} \wedge f_{94} \wedge f_{29} \wedge f_{7} \wedge f_{91} \wedge f_{47} \wedge f_{79} \wedge f_{92} \wedge f_{73} \wedge f_{54} \wedge f_{67} \wedge f_{97} \wedge f_{89} \wedge f_{98}) $$
    


    Number of Samples = 320
    Error = 35
    $$ (f_{4} \wedge f_{41} \wedge f_{1} \wedge f_{89} \wedge f_{16} \wedge f_{87} \wedge f_{77} \wedge f_{13} \wedge f_{55} \wedge f_{56} \wedge f_{63} \wedge f_{92} \wedge f_{76} \wedge f_{93} \wedge f_{66} \wedge f_{96} \wedge f_{99} \wedge f_{95}) \vee (f_{4} \wedge f_{41} \wedge f_{97} \wedge f_{91} \wedge f_{43} \wedge f_{39} \wedge f_{47} \wedge f_{64} \wedge f_{72} \wedge f_{13} \wedge f_{9} \wedge f_{96} \wedge f_{90} \wedge f_{99} \wedge f_{80} \wedge f_{89}) \vee (f_{4} \wedge f_{41} \wedge f_{97} \wedge f_{91} \wedge f_{85} \wedge f_{56} \wedge f_{99} \wedge f_{38} \wedge f_{66} \wedge f_{48} \wedge f_{16} \wedge f_{88} \wedge f_{100} \wedge f_{90} \wedge f_{95}) \vee (f_{4} \wedge f_{41} \wedge f_{97} \wedge f_{91} \wedge f_{89} \wedge f_{69} \wedge f_{38} \wedge f_{15} \wedge f_{87} \wedge f_{49} \wedge f_{28} \wedge f_{26} \wedge f_{90} \wedge f_{78} \wedge f_{93} \wedge f_{95}) \vee (f_{4} \wedge f_{41} \wedge f_{39} \wedge f_{1} \wedge f_{25} \wedge f_{56} \wedge f_{66} \wedge f_{53} \wedge f_{86} \wedge f_{96} \wedge f_{71} \wedge f_{98} \wedge f_{97}) \vee (f_{60} \wedge f_{99} \wedge f_{31} \wedge f_{1} \wedge f_{65} \wedge f_{5} \wedge f_{4} \wedge f_{44} \wedge f_{74} \wedge f_{48} \wedge f_{8} \wedge f_{91} \wedge f_{100} \wedge f_{97} \wedge f_{86}) \vee (f_{92} \wedge f_{41} \wedge f_{65} \wedge f_{3} \wedge f_{69} \wedge f_{49} \wedge f_{9} \wedge f_{71} \wedge f_{97} \wedge f_{96} \wedge f_{100} \wedge f_{98} \wedge f_{95} \wedge f_{99}) \vee (f_{92} \wedge f_{41} \wedge f_{76} \wedge f_{3} \wedge f_{61} \wedge f_{18} \wedge f_{77} \wedge f_{16} \wedge f_{58} \wedge f_{83} \wedge f_{60} \wedge f_{89} \wedge f_{99} \wedge f_{97} \wedge f_{94}) \vee (f_{92} \wedge f_{41} \wedge f_{76} \wedge f_{3} \wedge f_{61} \wedge f_{18} \wedge f_{77} \wedge f_{67} \wedge f_{99} \wedge f_{53} \wedge f_{100} \wedge f_{90} \wedge f_{88} \wedge f_{69}) $$
    

In [ ]:
from matplotlib import pyplot as plt

plt.plot(samples, errors, color='c')
plt.xlabel('Number of samples')
plt.ylabel('Error')
plt.grid()
plt.show()